# Volatility Smile and Term Structure Analysis

This notebook demonstrates how to analyze volatility surfaces using implied volatility.

## What is a Volatility Smile?

The **volatility smile** is a pattern where implied volatilities vary with strike price. In the Black-Scholes model, volatility is assumed constant across all strikes. However, market prices often imply different volatilities for different strikes:

- **At-the-money (ATM)** options typically have lower implied volatility
- **Out-of-the-money (OTM)** puts and calls often have higher implied volatility

This creates a "smile" or "smirk" shape when plotting IV vs strike.

### Why Do Smiles Exist?

1. **Fat tails**: Real asset returns have fatter tails than the normal distribution assumes
2. **Jump risk**: Markets can experience sudden large moves
3. **Supply/demand**: OTM puts are popular for hedging, driving up their prices
4. **Leverage effect**: Volatility often increases when prices fall

## Setup

In [ ]:
import sys
sys.path.insert(0, '..')

import matplotlib.pyplot as plt
import numpy as np

from src.analysis.volatility_smile import (
    generate_vol_smile_data,
    generate_term_structure_data,
)

# Plot styling
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

## Part 1: Volatility Smile

We'll generate synthetic option prices using Black-Scholes with a flat volatility, then recover implied volatilities.

**Theory**: If we use a constant volatility to generate prices, we should recover a flat "smile" (not really a smile at all).

In [ ]:
# Market parameters
spot = 100
rate = 0.02
T = 0.5  # 6 months to maturity
true_vol = 0.20  # 20% volatility

# Strike grid
strikes = [70, 80, 90, 100, 110, 120, 130]

print("Parameters:")
print(f"  Spot:       ${spot}")
print(f"  Rate:       {rate:.1%}")
print(f"  Maturity:   {T} years")
print(f"  True Vol:   {true_vol:.1%}")
print(f"  Strikes:    {strikes}")

In [ ]:
# Generate clean smile (no noise)
strikes_clean, ivs_clean = generate_vol_smile_data(
    spot, rate, T, strikes, true_vol,
    apply_noise=False
)

# Generate noisy smile (simulating market data)
strikes_noisy, ivs_noisy = generate_vol_smile_data(
    spot, rate, T, strikes, true_vol,
    apply_noise=True,
    noise_std=0.05,
    seed=42
)

print("Implied Volatilities:")
print(f"{'Strike':<10} {'Clean IV':<12} {'Noisy IV':<12} {'Difference'}")
print("-" * 45)
for K, iv_c, iv_n in zip(strikes, ivs_clean, ivs_noisy):
    diff = (iv_n - iv_c) * 100  # In percentage points
    print(f"{K:<10} {iv_c:.4f}       {iv_n:.4f}       {diff:+.2f}pp")

In [ ]:
# Plot the volatility smile
fig, ax = plt.subplots(figsize=(10, 6))

# Plot clean and noisy smiles
ax.plot(strikes_clean, [iv * 100 for iv in ivs_clean], 
        'b-o', linewidth=2, markersize=8, label='Clean (Flat Vol)')
ax.plot(strikes_noisy, [iv * 100 for iv in ivs_noisy], 
        'r--s', linewidth=2, markersize=8, alpha=0.7, label='Noisy (Market-like)')

# Reference line at true volatility
ax.axhline(y=true_vol * 100, color='gray', linestyle=':', 
           linewidth=1.5, label=f'True Vol ({true_vol:.0%})')

# Mark ATM region
ax.axvline(x=spot, color='green', linestyle='--', alpha=0.5, label='ATM')

ax.set_xlabel('Strike Price ($)', fontsize=12)
ax.set_ylabel('Implied Volatility (%)', fontsize=12)
ax.set_title('Implied Volatility Smile', fontsize=14)
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)

# Set y-axis limits for better visualization
ax.set_ylim([15, 30])

plt.tight_layout()
plt.show()

### Observations

- **Clean smile**: When we use a constant volatility to generate prices, the recovered implied volatilities are exactly equal to the true volatility (flat line at 20%).

- **Noisy smile**: When we add random noise to prices (simulating market imperfections), the recovered IVs show variation around the true value. In real markets, this variation follows patterns (smile/skew) due to the reasons mentioned above.

### How IV is Recovered

For each strike, we:
1. Observe the market price
2. Use a root-finding algorithm (Brent's method) to find σ such that BS(σ) = market price
3. This σ is the implied volatility

## Part 2: Volatility Term Structure

The **volatility term structure** shows how implied volatility varies with time to maturity.

Common patterns:
- **Upward sloping**: Short-term vol < Long-term vol (normal markets)
- **Inverted**: Short-term vol > Long-term vol (during crises)
- **Humped**: Peak at intermediate maturities

In [ ]:
# Term structure parameters
strike = 100  # ATM strike
maturities = [0.25, 0.5, 1.0, 2.0]  # 3mo, 6mo, 1yr, 2yr

# Upward-sloping term structure (typical)
true_vols_upward = [0.18, 0.20, 0.22, 0.25]

# Generate term structure data
mats_out, ivs_term = generate_term_structure_data(
    spot, strike, rate, maturities, true_vols_upward
)

print("Volatility Term Structure:")
print(f"{'Maturity (yrs)':<15} {'True Vol':<12} {'Recovered IV'}")
print("-" * 40)
for T, vol, iv in zip(maturities, true_vols_upward, ivs_term):
    print(f"{T:<15} {vol:.2%}         {iv:.4f}")

In [ ]:
# Plot term structure
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(mats_out, [iv * 100 for iv in ivs_term], 
        'b-o', linewidth=2, markersize=10)

ax.set_xlabel('Time to Maturity (Years)', fontsize=12)
ax.set_ylabel('Implied Volatility (%)', fontsize=12)
ax.set_title('Implied Volatility Term Structure', fontsize=14)
ax.grid(True, alpha=0.3)

# Add annotations
for T, iv in zip(mats_out, ivs_term):
    ax.annotate(f'{iv:.1%}', (T, iv * 100), 
                textcoords="offset points", 
                xytext=(0, 10), ha='center', fontsize=9)

plt.tight_layout()
plt.show()

### Interpreting the Term Structure

The upward-sloping term structure we've modeled suggests:

- **Short-term (3mo)**: 18% volatility - lower uncertainty in near term
- **Long-term (2yr)**: 25% volatility - higher uncertainty over longer horizons

This is typical during calm market conditions where long-term uncertainty exceeds short-term.

### Applications

Understanding volatility surfaces is crucial for:
- **Pricing exotic options** that depend on the full surface
- **Risk management** to capture smile/skew risk
- **Trading strategies** like volatility arbitrage
- **Model calibration** for stochastic volatility models (Heston, SABR)

## Summary

| Concept | Description | Key Insight |
|---------|-------------|-------------|
| Implied Volatility | The vol that makes BS price match market | Backed out via root-finding |
| Volatility Smile | IV varies with strike | OTM options often have higher IV |
| Term Structure | IV varies with maturity | Reflects uncertainty over time |

In real markets, the full volatility surface (IV vs strike vs maturity) is not flat, and modeling this surface accurately is a key challenge in quantitative finance.